In [10]:
import pandas as pd
import numpy as np
import json # library to handle JSON files
import sys
!{sys.executable} -m pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!{sys.executable} -m pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Creating a dataframe from the file saved in the first part of the project.

In [11]:
# The code was removed by Watson Studio for sharing.

In [12]:
# The code was removed by Watson Studio for sharing.

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rougetd,Malverntd",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creektd,Rouge Hilltd,Port Uniontd",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningsidetd,West Hilltd",43.763573,-79.188711
3,M1G,Scarborough,Woburntd,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Use geopy library to get the latitude and longitude values of Toronto

In [13]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="Torinto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top.

In [14]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood , borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Define Foursquare Credentials and Version

In [15]:
# The code was removed by Watson Studio for sharing.

Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.

In [16]:
df.loc[0, 'Neighbourhood']

'Rougetd,Malverntd'

In [17]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rougetd,Malverntd are 43.8066863, -79.19435340000003.


Now, let's get the top 100 venues that are in Rougetd,Malverntd within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [18]:
radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    'WKFZFZ02LNG0VH3REMIHILKIGDG51FRI1O2W0D3VBXLEKPB4', 
    'TPFU0G1EQ35MHQ2U252TXEEHTUXLP3NJXDPLPTRHO3OTFUEM', 
    '20180604', 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=WKFZFZ02LNG0VH3REMIHILKIGDG51FRI1O2W0D3VBXLEKPB4&client_secret=TPFU0G1EQ35MHQ2U252TXEEHTUXLP3NJXDPLPTRHO3OTFUEM&v=20180604&ll=43.8066863,-79.19435340000003&radius=500&limit=100'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d67962f66fc65002c8a88d5'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.811186304500005,
    'lng': -79.18812958073043},
   'sw': {'lat': 43.8021862955, 'lng': -79.20057721926962}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'di

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


And how many venues were returned by Foursquare?

In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


<h1>Explore Neighborhoods in Manhattan</h1>

Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called Toronto_venues.

In [24]:
Toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Rougetd,Malverntd
Highland Creektd,Rouge Hilltd,Port Uniontd
Guildwood,Morningsidetd,West Hilltd
Woburntd
Cedarbrae
Scarborough Villagetd
East Birchmount Park,Ionviewtd,Kennedy Parktd
Clairleatd,Golden Miletd,Oakridgetd
Cliffcresttd,Cliffsidetd,Scarborough Village West
Birch Clifftd,Cliffside West
Dorset Parktd,Scarborough Town Centretd,Wexford Heightstd
Maryvaletd,Wexfordtd
Agincourttd
Clarks Corners,Sullivan,Tam O'Shantertd
Agincourt Northtd,L'Amoreaux East,Millikentd,Steeles East
L'Amoreaux West
Upper Rougetd
Hillcrest Villagetd
Fairview,Henry Farmtd,Oriole
Bayview Villagetd
Silver Hills,York Millstd
Newtonbrooktd,Willowdaletd
Willowdale South
York Mills West
Willowdale Westtd
Parkwoodstd
Don Mills North
Flemingdon Parktd,Don Mills South
Bathurst Manortd,Downsview North,Wilson Heightstd
Northwood Parktd,York Universitytd
CFB Torontotd,Downsview East
Downsview Westtd
Downsview Central
Downsview Northwest
Victoria Villagetd
Woodbine Gardenstd,Parkview Hilltd
Woodbine Heightstd
The Bea

In [25]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2253, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rougetd,Malverntd",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creektd,Rouge Hilltd,Port Uniontd",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningsidetd,West Hilltd",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningsidetd,West Hilltd",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningsidetd,West Hilltd",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


Let's check how many venues were returned for each neighborhood

In [26]:
Toronto_groupued =Toronto_venues.groupby('Neighbourhood').count()
Toronto_groupued

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
"Agincourt Northtd,L'Amoreaux East,Millikentd,Steeles East",2,2,2,2,2,2
Agincourttd,4,4,4,4,4,4
"Albion Gardens,Beaumond Heightstd,Humbergate,Jamestowntd,Mount Olivetd,Silverstonetd,South Steelestd,Thistletowntd",10,10,10,10,10,10
"Alderwoodtd,Long Branchtd",8,8,8,8,8,8
"Bathurst Manortd,Downsview North,Wilson Heightstd",19,19,19,19,19,19
Bayview Villagetd,4,4,4,4,4,4
"Bedford Parktd,Lawrence Manor East",23,23,23,23,23,23
Berczy Parktd,56,56,56,56,56,56


Let's find out how many unique categories can be curated from all the returned venues¶

In [27]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


<h1>Analyze Each Neighborhood</h1>

In [28]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe

Toronto_onehot.shape
Toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = list(Toronto_venues['Neighbourhood'])
Toronto_onehot.shape
Toronto_onehot.head()
'Neighbourhood' in list(Toronto_onehot.columns)

True

In [30]:
# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

'Neighbourhood' in list(Toronto_onehot.columns)

True

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.0,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0,0.000000
1,"Agincourt Northtd,L'Amoreaux East,Millikentd,S...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,Agincourttd,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,"Albion Gardens,Beaumond Heightstd,Humbergate,J...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,"Alderwoodtd,Long Branchtd",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,"Bathurst Manortd,Downsview North,Wilson Heightstd",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,Bayview Villagetd,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,"Bedford Parktd,Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,Berczy Parktd,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,"Birch Clifftd,Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


Let's print each neighborhood along with the top 5 most common venues

In [32]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
            venue  freq
0     Coffee Shop  0.08
1            Café  0.05
2      Steakhouse  0.04
3             Bar  0.04
4  Cosmetics Shop  0.03


----Agincourt Northtd,L'Amoreaux East,Millikentd,Steeles East----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2    Accessories Store   0.0
3   Mexican Restaurant   0.0
4  Monument / Landmark   0.0


----Agincourttd----
               venue  freq
0     Breakfast Spot  0.25
1             Lounge  0.25
2     Sandwich Place  0.25
3       Skating Rink  0.25
4  Accessories Store  0.00


----Albion Gardens,Beaumond Heightstd,Humbergate,Jamestowntd,Mount Olivetd,Silverstonetd,South Steelestd,Thistletowntd----
            venue  freq
0   Grocery Store   0.2
1     Pizza Place   0.1
2     Coffee Shop   0.1
3  Discount Store   0.1
4  Sandwich Place   0.1


----Alderwoodtd,Long Branchtd----
            venue  freq
0     Pizza Place  0.25
1             Gym  0.12
2     Coffee Shop  0.1

Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Bar,Burger Joint,Thai Restaurant,Cosmetics Shop,Hotel,Restaurant,Asian Restaurant
1,"Agincourt Northtd,L'Amoreaux East,Millikentd,S...",Park,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,Agincourttd,Lounge,Skating Rink,Breakfast Spot,Sandwich Place,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
3,"Albion Gardens,Beaumond Heightstd,Humbergate,J...",Grocery Store,Beer Store,Fried Chicken Joint,Pharmacy,Pizza Place,Coffee Shop,Fast Food Restaurant,Discount Store,Sandwich Place,Doner Restaurant
4,"Alderwoodtd,Long Branchtd",Pizza Place,Gym,Coffee Shop,Pharmacy,Skating Rink,Sandwich Place,Pub,Dog Run,Dessert Shop,Dim Sum Restaurant


<h1>Cluster Neighborhoods</h1>


Run k-means to cluster the neighborhood into 5 clusters.

In [35]:
# set number of clusters
kclusters =5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 2, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head(15) # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rougetd,Malverntd",43.806686,-79.194353,0.0,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Rec Center
1,M1C,Scarborough,"Highland Creektd,Rouge Hilltd,Port Uniontd",43.784535,-79.160497,4.0,Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant
2,M1E,Scarborough,"Guildwood,Morningsidetd,West Hilltd",43.763573,-79.188711,0.0,Electronics Store,Mexican Restaurant,Rental Car Location,Pizza Place,Breakfast Spot,Intersection,Medical Center,Yoga Studio,Diner,Discount Store
3,M1G,Scarborough,Woburntd,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Caribbean Restaurant,Bank,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Bakery,Eastern European Restaurant,Dumpling Restaurant,Drugstore
5,M1J,Scarborough,Scarborough Villagetd,43.744734,-79.239476,2.0,Spa,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
6,M1K,Scarborough,"East Birchmount Park,Ionviewtd,Kennedy Parktd",43.727929,-79.262029,0.0,Department Store,Discount Store,Bus Station,Coffee Shop,Hobby Shop,Convenience Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
7,M1L,Scarborough,"Clairleatd,Golden Miletd,Oakridgetd",43.711112,-79.284577,0.0,Bakery,Bus Line,Fast Food Restaurant,Park,Metro Station,Bus Station,Intersection,Soccer Field,Coworking Space,Doner Restaurant
8,M1M,Scarborough,"Cliffcresttd,Cliffsidetd,Scarborough Village West",43.716316,-79.239476,0.0,Motel,Skating Rink,American Restaurant,Ethiopian Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store
9,M1N,Scarborough,"Birch Clifftd,Cliffside West",43.692657,-79.264848,0.0,College Stadium,General Entertainment,Skating Rink,Café,Comic Shop,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Colombian Restaurant


In [37]:
Toronto_merged =Toronto_merged[Toronto_merged['Cluster Labels'].isnull()!=True]
Toronto_merged['Cluster Labels']= Toronto_merged['Cluster Labels'].astype(int)
Toronto_merged['Cluster Labels'].values

array([0, 4, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       3, 0, 0, 0, 0, 3, 1, 0, 1, 0, 0])

Finally, let's visualize the resulting clusters

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1>Examine Clusters</h1>

In [39]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Rec Center
2,Scarborough,0,Electronics Store,Mexican Restaurant,Rental Car Location,Pizza Place,Breakfast Spot,Intersection,Medical Center,Yoga Studio,Diner,Discount Store
3,Scarborough,0,Coffee Shop,Korean Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,Scarborough,0,Hakka Restaurant,Caribbean Restaurant,Bank,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Bakery,Eastern European Restaurant,Dumpling Restaurant,Drugstore
6,Scarborough,0,Department Store,Discount Store,Bus Station,Coffee Shop,Hobby Shop,Convenience Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
7,Scarborough,0,Bakery,Bus Line,Fast Food Restaurant,Park,Metro Station,Bus Station,Intersection,Soccer Field,Coworking Space,Doner Restaurant
8,Scarborough,0,Motel,Skating Rink,American Restaurant,Ethiopian Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store
9,Scarborough,0,College Stadium,General Entertainment,Skating Rink,Café,Comic Shop,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Colombian Restaurant
10,Scarborough,0,Indian Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Chinese Restaurant,Pet Store,Latin American Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
11,Scarborough,0,Smoke Shop,Bakery,Breakfast Spot,Middle Eastern Restaurant,Yoga Studio,Drugstore,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant


In [40]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,North York,1,Bus Stop,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
30,North York,1,Airport,Park,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,Central Toronto,1,Park,Swim School,Bus Line,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
50,Downtown Toronto,1,Park,Trail,Playground,Building,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
74,York,1,Park,Women's Store,Market,Fast Food Restaurant,Grocery Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
90,Etobicoke,1,River,Park,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
98,York,1,Park,Convenience Store,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
100,Etobicoke,1,Park,Pizza Place,Mobile Phone Shop,Bus Line,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant


In [41]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2,Spa,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
14,Scarborough,2,Park,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
48,Central Toronto,2,Playground,Yoga Studio,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [42]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,North York,3,Baseball Field,Food Truck,Home Service,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
63,Central Toronto,3,Garden,Home Service,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
91,Etobicoke,3,Baseball Field,Pool,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
97,North York,3,Baseball Field,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant


In [43]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4,Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant
